In [1]:
import codenamesLLM
import pandas as pd
import openpyxl
from tqdm import tqdm
import xlsxwriter
from random import shuffle

c:\Users\frmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = r"experiment_data\mixed_guessers_data\mixed_guessers_input_2.xlsx"
df = pd.read_excel(file_path)
total_games = len(df)

for index, row in df.iterrows():
    red_guesser_team = [row['rg1'], row['rg2'], row['rg3']]
    red_guesser_label = "mixed" if (red_guesser_team[0] != red_guesser_team[1]) else red_guesser_team[0]
    blue_guesser_team = [row['bg1'], row['bg2'], row['bg3']]
    blue_guesser_label = "mixed" if (blue_guesser_team[0] != blue_guesser_team[1]) else blue_guesser_team[0]
    shuffle(red_guesser_team)
    shuffle(blue_guesser_team)

    playable = True
    already_played = pd.notna(row['winner'])

    if not(already_played) and playable:
        try:
            print(f"playing {red_guesser_label} vs {blue_guesser_label} ({index}/{total_games})")
            result = codenamesLLM.play_game_mixed_guessers(red_master_model="grok-beta",
                                                        red_guesser_model=red_guesser_team,
                                                        blue_master_model="grok-beta",
                                                        blue_guesser_model=blue_guesser_team)
            
            print(result)
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_model'] = red_guesser_label
            df.at[index, 'blue_model'] = blue_guesser_label
            df.at[index, 'winner'] = result[0]
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

            df.to_excel(file_path, index=False)
            print("game saved.")
        
        except Exception as e:
            print(f"skipped game: {e}")

playing claude-3-5-sonnet-latest vs mixed (0/96)
('BLUE', 'cards finished', 12, ['Team RED spymaster said: EUROPE (2).', 'Team RED said: FRANCE. The word was RED.', 'Team RED said: LONDON. The word was RED.', 'Team BLUE spymaster said: LEADER (1).', 'Team BLUE said: PILOT. The word was BLUE.', 'Team RED spymaster said: CROWN (3).', 'Team RED said: COURT. The word was neutral.', 'Team BLUE spymaster said: ZODIAC (1).', 'Team BLUE said: SCORPION. The word was BLUE.', 'Team RED spymaster said: TIP (2).', 'Team RED said: POINT. The word was RED.', 'Team RED said: SPIKE. The word was RED.', 'Team BLUE spymaster said: GAMBLE (2).', 'Team BLUE said: ROULETTE. The word was BLUE.', 'Team BLUE said: DICE. The word was neutral.', 'Team RED spymaster said: CAVE (1).', 'Team RED said: ROCK. The word was neutral.', 'Team BLUE spymaster said: POST (2).', 'Team BLUE said: MAIL. The word was BLUE.', 'Team BLUE said: PORT. The word was neutral.', 'Team RED spymaster said: DRAMA (2).', 'Team RED said: TH

# PLAY AGGREGATE GAMES BEST MODEL

In [ ]:
# Path to your Excel file
file_path = r"experiment_data\best_model\best_model.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_team = row['red_model']  # Replace with the actual column title for the red team
    blue_team = row['blue_model']  # Replace with the actual column title for the blue team

    if red_team == "grok-claude":
        red_master_model = "grok-beta"
        red_guesser_model = "claude-3-5-sonnet-latest"
    else:
        red_master_model = red_team
        red_guesser_model = red_team

    if blue_team == "grok-claude":
        blue_master_model = "grok-beta"
        blue_guesser_model = "claude-3-5-sonnet-latest"
    else:
        blue_master_model = blue_team
        blue_guesser_model = blue_team

    playable = True
    already_played = pd.notna(row['winner'])

    if ((pd.notna(red_team) and pd.notna(blue_team)) and not(already_played)) and playable:  # Check if both values are not NaN
        try:
            # Call your function and get the result
            print(f"playing {red_team} vs {blue_team}...")
            result = codenamesLLM.play_game(red_master_model=red_master_model,
                                            red_guesser_model=red_guesser_model,
                                            blue_master_model=blue_master_model,
                                            blue_guesser_model=blue_guesser_model)
            
            print(result)
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_model'] = red_team
            df.at[index, 'blue_model'] = blue_team
            df.at[index, 'winner'] = result[0]
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

        except Exception as e:
            print(f"skipped game: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)

# TOURNAMENT DATA PROCESS

In [3]:
import pandas as pd

def process_tournament_data(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group):
        wins = group['winner'] == group['role'].str.upper()
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "model_name": group["model"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group['avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_ending_cards": group.loc[wins & card_finished, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_ending_cards": group.loc[losses & card_finished, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_killer_card": group.loc[wins & killer_word, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_killer_card": group.loc[losses & killer_word, 'avg_words_2guess'].mean(),
            "average_word_guessed": group['avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_ending_cards": group.loc[wins & card_finished, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_ending_cards": group.loc[losses & card_finished, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_killer_card": group.loc[wins & killer_word, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_killer_card": group.loc[losses & killer_word, 'avg_words_guessed'].mean(),
            "average_turns": group['turns'].mean(),
            "average_turns_when_wins": group.loc[wins, 'turns'].mean(),
            "average_turns_when_lose": group.loc[losses, 'turns'].mean(),
            "average_turns_when_wins_by_ending_cards": group.loc[wins & card_finished, 'turns'].mean(),
            "average_turns_when_loses_by_ending_cards": group.loc[losses & card_finished, 'turns'].mean(),
            "average_turns_when_wins_by_killer_cards": group.loc[wins & killer_word, 'turns'].mean(),
            "average_turns_when_loses_by_killer_cards": group.loc[losses & killer_word, 'turns'].mean(),
            "total_cib": group['cib'].sum()
        }

    # Reshape the dataset to treat roles equivalently
    red_df = df.rename(columns=lambda x: x.replace('red_', '')).assign(role='red', model=df['red_model'])
    blue_df = df.rename(columns=lambda x: x.replace('blue_', '')).assign(role='blue', model=df['blue_model'])
    combined_df = pd.concat([red_df, blue_df], ignore_index=True)

    # Metrics for models as a whole
    overall_metrics = combined_df.groupby("model").apply(calculate_metrics).apply(pd.Series)

    # Metrics for models playing as Red
    red_metrics = red_df.groupby("model").apply(calculate_metrics).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for models playing as Blue
    blue_metrics = blue_df.groupby("model").apply(calculate_metrics).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)


In [ ]:
for model in ["vsclaude", "vsgpt", "vsgrok"]:
    input_path = r"C:\Users\miche\OneDrive\Desktop\codenames data backup\bestmodel\{}.xlsx".format(model)
    output_path = r"C:\Users\miche\OneDrive\Desktop\codenames data backup\bestmodel\{}_stats.xlsx".format(model)

    process_tournament_data(input_path, output_path)

# COT DATA PROCESS

In [ ]:
# Load the data from the uploaded file
file_path = 'experiment_data\cot_data\cot_data_new_prompt.xlsx'
df = pd.read_excel(file_path)
i = 0
# Loop through each configuration in the data file
for index, row in df.iterrows():
    i += 1
    red_cot = row['red_cot']
    blue_cot = row['blue_cot']
    playable = True
    already_played = pd.notna(row['winner'])
    # Call the play_game function with parameters from the row

    if not already_played and playable:  # Check if both values are not NaN
        try:

            print(f"playing game {i}")
            result = codenamesLLM.play_game(lang="eng", n_cards=25, coloured_cards=7, k_cards=1, verbose=False, 
              red_master_model="gpt-4o-mini", red_guesser_model=False, 
              red_cot=red_cot, red_cot_guesser=False, 
              blue_master_model="gpt-4o-mini", blue_guesser_model=False, 
              blue_cot=blue_cot, blue_cot_guesser=False, 
              red_agents=3, blue_agents=3, masterverbose=False)

            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'winner'] = result[0]
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]    
        except Exception as e:
            print(f"skipped game: {e}")
    df.to_excel('experiment_data\cot_data\cot_data_new_prompt.xlsx', index=False)

print(f"Results saved to experiment_data\cot_data\cot_data_new_prompt.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:40: SyntaxWarning: invalid escape sequence '\c'
<>:42: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:40: SyntaxWarning: invalid escape sequence '\c'
<>:42: SyntaxWarning: invalid escape sequence '\c'
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\1659186335.py:2: SyntaxWarning: invalid escape sequence '\c'
  file_path = 'experiment_data\cot_data\cot_data_new_prompt.xlsx'
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\1659186335.py:40: SyntaxWarning: invalid escape sequence '\c'
  df.to_excel('experiment_data\cot_data\cot_data_fra.xlsx', index=False)
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\1659186335.py:42: SyntaxWarning: invalid escape sequence '\c'
  print(f"Results saved to experiment_data\cot_data\cot_data_fra.xlsx")


Results saved to experiment_data\cot_data\cot_data_fra.xlsx


In [4]:
import pandas as pd

def process_tournament_data_by_cot(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group):
        wins = group['winner'] == group['role'].str.upper()
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "cot": group["cot"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group['avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, 'avg_words_2guess'].mean(),
            "average_word_guessed": group['avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, 'avg_words_guessed'].mean(),
            "average_turns": group['turns'].mean(),
            "average_turns_when_wins": group.loc[wins, 'turns'].mean(),
            "average_turns_when_lose": group.loc[losses, 'turns'].mean(),
            "total_cib": group['cib'].sum()
        }

    # Reshape the dataset to unify red and blue roles
    red_df = df.rename(columns=lambda x: x.replace('red_', '')).assign(role='red', cot=df['red_cot'])
    blue_df = df.rename(columns=lambda x: x.replace('blue_', '')).assign(role='blue', cot=df['blue_cot'])
    combined_df = pd.concat([red_df, blue_df], ignore_index=True)

    # Metrics for cot
    overall_metrics = combined_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)

    # Metrics for cot in Red role
    red_metrics = red_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for cot in Blue role
    blue_metrics = blue_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)


In [5]:
# Example usage:
input_path = 'experiment_data\cot_data\cot_data_new_prompt.xlsx'
output_path = 'experiment_data\cot_data\cot_data_new_prompt1.xlsx'
process_tournament_data_by_cot(input_path, output_path)

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\2757196823.py:2: SyntaxWarning: invalid escape sequence '\c'
  input_path = 'experiment_data\cot_data\cot_data_new_prompt.xlsx'
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\2757196823.py:3: SyntaxWarning: invalid escape sequence '\c'
  output_path = 'experiment_data\cot_data\cot_data_new_prompt1.xlsx'
C:\Users\frmar\AppData\Local\Temp\ipykernel_960\2111975892.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  overall_metrics = combined_df.groupby("

# AGENTS NUMBER DATA PROCESS


In [ ]:
import pandas as pd

def process_agents_data(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group, role):
        prefix = 'red_' if role == 'red' else 'blue_'
        wins = (group['winner'] == role.upper())
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "number_of_agents": group[f"number_{role}_agents"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group[f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_killer_card": group.loc[wins & killer_word, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_killer_card": group.loc[losses & killer_word, f'{prefix}avg_words_2guess'].mean(),
            "average_word_guessed": group[f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_killer_card": group.loc[wins & killer_word, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_killer_card": group.loc[losses & killer_word, f'{prefix}avg_words_guessed'].mean(),
            "average_turns": group[f'{prefix}turns'].mean(),
            "average_turns_when_wins": group.loc[wins, f'{prefix}turns'].mean(),
            "average_turns_when_lose": group.loc[losses, f'{prefix}turns'].mean(),
            "average_turns_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}turns'].mean(),
            "average_turns_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}turns'].mean(),
            "average_turns_when_wins_by_killer_cards": group.loc[wins & killer_word, f'{prefix}turns'].mean(),
            "average_turns_when_loses_by_killer_cards": group.loc[losses & killer_word, f'{prefix}turns'].mean(),
            "total_cib": group[f'{prefix}cib'].sum()
        }

    # Metrics for agents playing as Red
    red_metrics = df.groupby("number_red_agents").apply(lambda g: calculate_metrics(g, "red")).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for agents playing as Blue
    blue_metrics = df.groupby("number_blue_agents").apply(lambda g: calculate_metrics(g, "blue")).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Combine and calculate overall metrics
    overall_metrics = (
        pd.concat([red_metrics, blue_metrics])
        .groupby("number_of_agents")
        .agg({
            "games_played": "sum",
            "wins": "sum",
            "win_percentage": "mean",
            "win_by_cards_finished": "sum",
            "wins_by_killer_words": "sum",
            "losses_by_card_finished": "sum",
            "losses_by_killer_words": "sum",
            "average_word_to_guess": "mean",
            "average_word_to_guess_when_wins": "mean",
            "average_word_to_guess_when_lose": "mean",
            "average_word_to_guess_when_wins_by_ending_cards": "mean",
            "average_word_to_guess_when_loses_by_ending_cards": "mean",
            "average_word_to_guess_when_wins_by_killer_card": "mean",
            "average_word_to_guess_when_loses_by_killer_card": "mean",
            "average_word_guessed": "mean",
            "average_word_guessed_when_wins": "mean",
            "average_word_guessed_when_lose": "mean",
            "average_word_guessed_when_wins_by_ending_cards": "mean",
            "average_word_guessed_when_loses_by_ending_cards": "mean",
            "average_word_guessed_when_wins_by_killer_card": "mean",
            "average_word_guessed_when_loses_by_killer_card": "mean",
            "average_turns": "mean",
            "average_turns_when_wins": "mean",
            "average_turns_when_lose": "mean",
            "average_turns_when_wins_by_ending_cards": "mean",
            "average_turns_when_loses_by_ending_cards": "mean",
            "average_turns_when_wins_by_killer_cards": "mean",
            "average_turns_when_loses_by_killer_cards": "mean",
            "total_cib": "sum"
        })
        .reset_index()
    )

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)

In [8]:
# Define input and output file paths
input_path = r'experiment_data\n_agents_data\agents_data.xlsx'  # Replace with your input file path
output_path = r'experiment_data\n_agents_data\agents_data_stats.xlsx'  # Replace with your output file path

# Run the process
process_agents_data(input_path, output_path)

C:\Users\miche\AppData\Local\Temp\ipykernel_22320\2518959492.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  red_metrics = df.groupby("number_red_agents").apply(lambda g: calculate_metrics(g, "red")).apply(pd.Series)
C:\Users\miche\AppData\Local\Temp\ipykernel_22320\2518959492.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  blue_metrics = df.groupby("number_blue_agents").apply(lambda g: calculate_